In [2]:
from lightning_vocoders.models.hifigan.lightning_module import HiFiGANLightningModule
lightning_module = HiFiGANLightningModule.load_from_checkpoint("../tb_logs/lightning_logs/version_159/checkpoints/epoch=9-step=29480.ckpt")
from lightning_vocoders.models.hifigan.hifigan import Generator


/home/wnakata/lightning-vocoders/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/wnakata/lightning-vocoders/.venv/lib/python3.10/site-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2Model: ['quantizer.codevectors', 'quantizer.weight_proj.bias', 'project_q.weight', 'project_hid.bias', 'quantizer.weight_proj.weight', 'project_hid.weight', 'project_q.bias']
- This IS expected if you are initiali

In [3]:
generator_cfg = lightning_module.cfg.model.generator
generator = Generator(generator_cfg)

In [4]:
generator_cfg

{'num_input_channels': 768, 'upsample_rates': [7, 7, 3, 3], 'upsample_initial_channel': 512, 'upsample_kernel_sizes': [15, 15, 7, 7], 'resblock_dilation_sizes': [[1, 3, 5], [1, 3, 5], [1, 3, 5]], 'resblock_kernel_sizes': [3, 7, 11], 'resblock': '1'}

In [5]:
import torch
generator_cfg.upsample_rates = [7,7,3,3]
generator_cfg.upsample_kernel_sizes = [15,15,7,7]
generator = Generator(generator_cfg)
generator(torch.randn((1,50,768))).size()

torch.Size([1, 1, 22050])

In [9]:
from tqdm import tqdm
lengths = []
lightning_module.preprocessor.ssl_model.eval()
for i in range(2,50):
    start = 16_000//50 * i
    for j in tqdm(range(start,17_000)):
        if lightning_module.preprocessor.ssl_model(input_values=torch.randn(1,j).cuda()).last_hidden_state.size(1) == i:
            lengths.append(j )
            break

  0%|          | 10/11240 [00:00<14:13, 13.16it/s]


KeyboardInterrupt: 

In [10]:
lengths

[720,
 1040,
 1360,
 1680,
 2000,
 2320,
 2640,
 2960,
 3280,
 3600,
 3920,
 4240,
 4560,
 4880,
 5200,
 5520]

In [12]:
def get_length(x):
    return 16_000//50 * x + 80
[get_length(i) for i in range(10)]

[80, 400, 720, 1040, 1360, 1680, 2000, 2320, 2640, 2960]

In [22]:
lightning_module.preprocessor.ssl_model(input_values=torch.randn(1,get_length(49)).cuda()).last_hidden_state.size(1)

49

In [ ]:
ge

In [ ]:
from matplotlib import pyplot as plt
plt.plot(lengths)

In [ ]:
lightning_module.preprocessor.ssl_prepreocessor(torch.randn(16_000))

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


{'input_values': [array([ 0.15781322, -1.3593025 ,  0.07017981, ...,  0.6999978 ,
        0.65397877,  0.6198929 ], dtype=float32)]}

In [ ]:
generator_cfg.upsample_rates = [8,8,2,2]
generator_cfg.upsample_kernel_sizes = [16,16,8,8]
generator = Generator(generator_cfg)
generator(torch.randn((1,86,80))).size()

torch.Size([1, 1, 22016])

In [ ]:
generator_cfg.upsample_rates = [5,4,4,2,2,2]
generator_cfg.upsample_kernel_sizes = [11,8,8,4,4,4]
generator = Generator(generator_cfg)
generator(torch.randn((1,50,80))).size()

torch.Size([1, 1, 32000])